In [4]:
import datasets
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModel
import torch
import numpy as np
import pandas as pd
import utils
import os
import json
from tqdm.auto import tqdm
import joblib

In [5]:
train_ds = datasets.load_dataset('HuggingFaceM4/FairFace', '1.25', split='train', verification_mode="no_checks")

In [50]:
validation_ds = datasets.load_dataset('HuggingFaceM4/FairFace', '1.25', split="validation", verification_mode="no_checks")
validation_ds = validation_ds.shuffle(seed=42)

In [29]:
reported_stats = {}

In [30]:
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
# Load model and pre-processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
model.to(device)
vision_model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
vision_model.to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [32]:
def get_embedding_and_zs(sample):

    # Age prediction
    inputs = processor(text=["text"]*len(sample), images=sample["image"], return_tensors="pt", padding=True).to(device)
    # outputs = model.get_image_features(**inputs)
    # Store embeddings - dim 512
    # sample["proj_embeddings"] = model.get_image_features(**inputs)
    outputs = model(**inputs)
    # Store embeddings - dim 512
    sample["proj_embeddings"] = outputs.image_embeds
    # # Reduce the age by 2
    # sample["age"] = [age - 2 for age in sample["age"]] # Since classes 0 and 1 have been deleted
    return sample

In [33]:
# train_ds = train_ds.map(get_embedding_and_zs, batched = True, batch_size=32)

In [34]:
# age_labels = np.array(train_ds["age"])

In [35]:
# np.bincount(age_labels)

In [36]:
# from sklearn.metrics import classification_report


# age_scaler = joblib.load("models/projected_scaler.joblib")
# age_clf = joblib.load("models/lr_clf_proj_age.joblib")
# X_train_scaled = age_scaler.transform(np.array(train_ds["proj_embeddings"]))
# y_train_preds = age_clf.predict(X_train_scaled)
# print(classification_report(age_labels, y_train_preds))

In [37]:
# np.save("clip_lr_age_preds_train_42.npy", y_train_preds)

In [38]:
# from sklearn.metrics import precision_recall_fscore_support

# precision, recall, f_score_weighted, _ = precision_recall_fscore_support(age_labels, y_train_preds, average='weighted')
# _, _, f_score_macro, _ = precision_recall_fscore_support(age_labels, y_train_preds, average='macro')
# _, _, f_score_micro, _ = precision_recall_fscore_support(age_labels, y_train_preds, average='micro')
# class_rep = classification_report( age_labels, y_train_preds, output_dict=True)
# print(f"Training set metrics - Age (CLIP + LR) \n" + "="*40)
# print(f"Accuracy: {class_rep['accuracy']:.4f} Precision: {precision:.4f}, Recall: {recall:.4f}, F-Score(Weighted): {f_score_weighted:.4f}, F-Score(Micro): {f_score_micro:.4f}, F-Score(Macro): {f_score_macro:.4f}")

In [51]:
validation_ds = validation_ds.map(get_embedding_and_zs, batched = True, batch_size=32)

Map:   0%|          | 0/10954 [00:00<?, ? examples/s]

In [52]:
# def adjust_ages(sample):
#     sample["age"] = [age - 2 for age in sample["age"]] # Since classes 0 and 1 have been deleted
#     return sample

In [53]:
# Get gender stats first
# 0 - Male, 1 - Female
gender_labels = np.array(validation_ds["gender"])
reported_stats["perc_fem_val"] = round(np.sum(gender_labels == 1) / len(gender_labels) * 100, ndigits=2) 
reported_stats["perc_mal_val"] = round(np.sum(gender_labels == 0) / len(gender_labels) * 100, ndigits=2) 

In [57]:
# Get gender predictions
gender_model, _, _ = utils._load_lr_classifiers()
gender_preds = utils._predict_gender(gender_model=gender_model, images=np.array(validation_ds["proj_embeddings"]))

In [58]:
gender_labels_pred = np.argmax(gender_preds, axis=1)
reported_stats["perc_fem_val_pred"] = round(np.sum(gender_labels_pred == 1) / len(gender_labels_pred) * 100, ndigits=2) 
reported_stats["perc_mal_val_pred"] = round(np.sum(gender_labels_pred == 0) / len(gender_labels_pred) * 100, ndigits=2) 

In [59]:
np.save("clip_lr_gender_preds_val_42.npy", gender_labels_pred)

In [43]:
reported_stats

{'perc_fem_val': 47.12,
 'perc_mal_val': 52.88,
 'perc_fem_val_pred': 47.39,
 'perc_mal_val_pred': 52.61}

In [44]:
round(np.sum(gender_labels_pred == gender_labels) / len(gender_labels), 2)

0.96

In [45]:
# Filter and adjust age data
# age_validation_ds = validation_ds.filter(lambda sample: sample["age"] not in {0, 1}).map(adjust_ages, batched = True, batch_size=32) # Filter out the first two classes

In [46]:
# 0,1,2,3,4,5 - Up to 50 & 6,7,8 - Over 50
age_labels = np.array(validation_ds["age"])
reported_stats["perc_ut50_val"] = round(np.sum(age_labels <= 5) / len(age_labels) * 100, ndigits=2) 
reported_stats["perc_o50_val"] = round(np.sum(age_labels >= 6) / len(age_labels) * 100, ndigits=2)

In [47]:
# Get age predictions
age_scaler, age_model = utils._load_age_model()
age_preds = utils._predict_age(age_scaler=age_scaler, age_model=age_model, images=np.array(validation_ds["proj_embeddings"]))

In [48]:
from sklearn.metrics import classification_report


age_scaler = joblib.load("models/projected_scaler.joblib")
age_clf = joblib.load("models/lr_clf_proj_age.joblib")
X_val_scaled = age_scaler.transform(np.array(validation_ds["proj_embeddings"]))
y_val_preds = age_clf.predict(X_val_scaled)
print(classification_report(age_labels, y_val_preds))

              precision    recall  f1-score   support

           0       0.73      0.71      0.72       199
           1       0.79      0.84      0.81      1356
           2       0.60      0.45      0.51      1181
           3       0.65      0.74      0.69      3300
           4       0.51      0.50      0.51      2330
           5       0.49      0.45      0.47      1353
           6       0.50      0.46      0.48       796
           7       0.48      0.48      0.48       321
           8       0.48      0.39      0.43       118

    accuracy                           0.60     10954
   macro avg       0.58      0.56      0.57     10954
weighted avg       0.60      0.60      0.60     10954



In [49]:
np.save("clip_lr_age_preds_val_42.npy", y_val_preds)

In [23]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f_score_weighted, _ = precision_recall_fscore_support(age_labels, y_val_preds, average='weighted')
_, _, f_score_macro, _ = precision_recall_fscore_support(age_labels, y_val_preds, average='macro')
_, _, f_score_micro, _ = precision_recall_fscore_support(age_labels, y_val_preds, average='micro')
class_rep = classification_report( age_labels, y_val_preds, output_dict=True)
print(f"Validation set metrics - Age (CLIP + LR) \n" + "="*40)
print(f"Accuracy: {class_rep['accuracy']:.4f} Precision: {precision:.4f}, Recall: {recall:.4f}, F-Score(Weighted): {f_score_weighted:.4f}, F-Score(Micro): {f_score_micro:.4f}, F-Score(Macro): {f_score_macro:.4f}")

Validation set metrics - Age (CLIP + LR) 
Accuracy: 0.6021 Precision: 0.5962, Recall: 0.6021, F-Score(Weighted): 0.5966, F-Score(Micro): 0.6021, F-Score(Macro): 0.5675


In [27]:
np.save("clip_lr_age_preds_val_42_sure.npy", y_val_preds)

In [25]:
np.load("clip_lr_age_preds_val_42.npy") == y_val_preds

array([ True,  True,  True, ...,  True,  True,  True])

In [26]:
np.unique(np.argmax(age_preds, axis=1), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64),
 array([ 193, 1447,  878, 3767, 2294, 1230,  730,  318,   97], dtype=int64))

In [20]:
round(np.sum(np.argmax(age_preds, axis=1) == age_labels) / len(age_labels), 4)

0.6013

In [21]:
age_labels_pred = []
for pred in tqdm(age_preds):
    # print(pred)
    max_sum_age = {
        "Up to 50": sum([p for p in pred[:-3]]),
        "Over 50": sum([p for p in pred[-3:]])
    }
    # print(max_sum_age)
    age_labels_pred.append(int(max_sum_age["Up to 50"] < max_sum_age["Over 50"]))

age_labels_pred = np.array(age_labels_pred)

  0%|          | 0/10954 [00:00<?, ?it/s]

In [22]:
# age_labels_pred = np.argmax(age_preds, axis=1)
reported_stats["perc_ut50_val_pred"] = round(np.sum(age_labels_pred == 0) / len(age_labels_pred) * 100, ndigits=2) 
reported_stats["perc_o50_val_pred"] = round(np.sum(age_labels_pred == 1) / len(age_labels_pred) * 100, ndigits=2)

In [27]:
# "Over 50"if dominant_age in {"50-59", "60-69", "more than 70"} else "Up to 50"
reported_stats["perc_ut50_direct_val_pred"] = round(np.sum(np.argmax(age_preds, axis=1) <= 5) / len(age_preds) * 100, ndigits=2)
reported_stats["perc_o50_direct_val_pred"] = round(np.sum(np.argmax(age_preds, axis=1) >= 6) / len(age_preds) * 100, ndigits=2) 

In [28]:
reported_stats

{'perc_fem_val': 47.12,
 'perc_mal_val': 52.88,
 'perc_fem_val_pred': 47.39,
 'perc_mal_val_pred': 52.61,
 'perc_ut50_val': 88.73,
 'perc_o50_val': 11.27,
 'perc_ut50_val_pred': 89.96,
 'perc_o50_val_pred': 10.04,
 'perc_ut50_direct_val_pred': 89.54,
 'perc_o50_direct_val_pred': 10.46}

In [29]:
if os.path.exists("black_panther_demography.csv"):
    df = pd.read_csv("black_panther_demography.csv", index_col=0)
    reported_stats["avg_conf_gender_blackpanther"] = round(df.gender.apply(lambda gender: max(json.loads(gender.replace("\'", "\"")).values())).mean(), ndigits=4)
    reported_stats["avg_conf_age_blackpanther"] = round(df.max_sum_age.apply(lambda max_sum_age: max(json.loads(max_sum_age.replace("\'", "\"")).values())).mean(), ndigits=4)
    if "direct_age_group" not in df.columns:
        df["direct_age_group"] = df["dominant_age"].apply(lambda dominant_age: "Over 50"if dominant_age in {"50-59", "60-69", "more than 70"} else "Up to 50")
        df.to_csv("black_panther_demography.csv")
    reported_stats["avg_conf_direct_age_blackpanther"] = round(df.age.apply(lambda age: max(json.loads(age.replace("\'", "\"")).values())).mean(), ndigits=4)

In [30]:
# df

In [31]:
if os.path.exists("mama_mia_demography.csv"):
    df = pd.read_csv("mama_mia_demography.csv", index_col=0)
    reported_stats["avg_conf_gender_mamamia"] = round(df.gender.apply(lambda gender: max(json.loads(gender.replace("\'", "\"")).values())).mean(), ndigits=4)
    reported_stats["avg_conf_age_mamamia"] = round(df.max_sum_age.apply(lambda max_sum_age: max(json.loads(max_sum_age.replace("\'", "\"")).values())).mean(), ndigits=4)
    if "direct_age_group" not in df.columns:
        df["direct_age_group"] = df["dominant_age"].apply(lambda dominant_age: "Over 50"if dominant_age in {"50-59", "60-69", "more than 70"} else "Up to 50")
        df.to_csv("mama_mia_demography.csv")
    reported_stats["avg_conf_direct_age_mamamia"] = round(df.age.apply(lambda age: max(json.loads(age.replace("\'", "\"")).values())).mean(), ndigits=4)

In [32]:
if os.path.exists("marigold_demography.csv"):
    df = pd.read_csv("marigold_demography.csv", index_col=0)
    reported_stats["avg_conf_gender_marigold"] = round(df.gender.apply(lambda gender: max(json.loads(gender.replace("\'", "\"")).values())).mean(), ndigits=4)
    reported_stats["avg_conf_age_marigold"] = round(df.max_sum_age.apply(lambda max_sum_age: max(json.loads(max_sum_age.replace("\'", "\"")).values())).mean(), ndigits=4)
    if "direct_age_group" not in df.columns:
        df["direct_age_group"] = df["dominant_age"].apply(lambda dominant_age: "Over 50"if dominant_age in {"50-59", "60-69", "more than 70"} else "Up to 50")
        df.to_csv("marigold_demography.csv")
    reported_stats["avg_conf_direct_age_marigold"] = round(df.age.apply(lambda age: max(json.loads(age.replace("\'", "\"")).values())).mean(), ndigits=4)

In [33]:
reported_stats

{'perc_fem_val': 47.12,
 'perc_mal_val': 52.88,
 'perc_fem_val_pred': 47.39,
 'perc_mal_val_pred': 52.61,
 'perc_ut50_val': 88.73,
 'perc_o50_val': 11.27,
 'perc_ut50_val_pred': 89.96,
 'perc_o50_val_pred': 10.04,
 'perc_ut50_direct_val_pred': 89.54,
 'perc_o50_direct_val_pred': 10.46,
 'avg_conf_gender_blackpanther': 0.9856,
 'avg_conf_age_blackpanther': 0.9211,
 'avg_conf_direct_age_blackpanther': 0.5519,
 'avg_conf_gender_mamamia': 0.9716,
 'avg_conf_age_mamamia': 0.8682,
 'avg_conf_direct_age_mamamia': 0.5138,
 'avg_conf_gender_marigold': 0.9564,
 'avg_conf_age_marigold': 0.8603,
 'avg_conf_direct_age_marigold': 0.53}

In [34]:
df = pd.DataFrame([reported_stats])

In [35]:
df

,perc_fem_val,perc_mal_val,perc_fem_val_pred,perc_mal_val_pred,perc_ut50_val,perc_o50_val,perc_ut50_val_pred,perc_o50_val_pred,perc_ut50_direct_val_pred,perc_o50_direct_val_pred,avg_conf_gender_blackpanther,avg_conf_age_blackpanther,avg_conf_direct_age_blackpanther,avg_conf_gender_mamamia,avg_conf_age_mamamia,avg_conf_direct_age_mamamia,avg_conf_gender_marigold,avg_conf_age_marigold,avg_conf_direct_age_marigold
0,47.12,52.88,47.39,52.61,88.73,11.27,89.96,10.04,89.54,10.46,0.9856,0.9211,0.5519,0.9716,0.8682,0.5138,0.9564,0.8603,0.53


In [36]:
df.to_csv("summary.csv", index=False)